In [1]:
import os
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator


In [2]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
training_set = train_datagen.flow_from_directory(
        'tranee_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

Found 250 images belonging to 5 classes.


In [3]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        'test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

Found 248 images belonging to 5 classes.


In [4]:
cnn = tf.keras.models.Sequential()

In [5]:
cnn.add(tf.keras.layers.Conv2D(filters=64 , kernel_size=3 , activation='relu' , input_shape=[64,64,3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=64 , kernel_size=3 , activation='relu' ))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2 , strides=2))

In [7]:
cnn.add(tf.keras.layers.Dropout(0.5))

In [8]:
cnn.add(tf.keras.layers.Flatten())

In [18]:
cnn.add(tf.keras.layers.Dense(units=64, activation='relu'),)

In [19]:
cnn.add(tf.keras.layers.Dense(units=32, activation='relu'),)

In [20]:
cnn.add(tf.keras.layers.Dense(units=5 , activation='softmax'))

In [12]:
cnn.compile(optimizer = 'rmsprop' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [21]:
cnn.fit(x = training_set , validation_data = test_set , epochs = 20)

Epoch 1/20
8/8 [==============================] - 15s 2s/step - loss: 0.2638 - accuracy: 0.9000 - val_loss: 0.7621 - val_accuracy: 0.7540
Epoch 2/20
8/8 [==============================] - 13s 2s/step - loss: 0.2717 - accuracy: 0.9040 - val_loss: 1.3928 - val_accuracy: 0.6774
Epoch 3/20
8/8 [==============================] - 13s 2s/step - loss: 0.2575 - accuracy: 0.9280 - val_loss: 0.6664 - val_accuracy: 0.7782
Epoch 4/20
8/8 [==============================] - 13s 2s/step - loss: 0.2208 - accuracy: 0.9120 - val_loss: 0.5524 - val_accuracy: 0.8105
Epoch 5/20
8/8 [==============================] - 13s 2s/step - loss: 0.3218 - accuracy: 0.8680 - val_loss: 0.6316 - val_accuracy: 0.7661
Epoch 6/20
8/8 [==============================] - 14s 2s/step - loss: 0.2050 - accuracy: 0.9000 - val_loss: 0.6572 - val_accuracy: 0.7702
Epoch 7/20
8/8 [==============================] - 13s 2s/step - loss: 0.2182 - accuracy: 0.9040 - val_loss: 0.6452 - val_accuracy: 0.7944
Epoch 8/20
8/8 [==================

In [14]:
tf.keras.models.save_model(cnn,'my_model.hdf5')

C:\Users\prana\AppData\Local\Temp\ipykernel_25480\3655746068.py:1: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(cnn,'my_model.hdf5')


In [15]:
!pip install streamlit

In [17]:

%%writefile app.py
import streamlit as st
import tensorflow as tf


@st.cache(allow_output_mutation=True)
def load_model():
  model=tf.keras.models.load_model('C:\\Users\\prana\\Desktop\\AI\AIML-test\\Flower_Reco\\my_model.hdf5')
  return model
with st.spinner('Model is being loaded..'):
  model=load_model()

st.write("""
         # Flower Classification
         """
         )

file = st.file_uploader("Please upload an brain scan file", type=["jpg", "png"])
import cv2
from PIL import Image, ImageOps
import numpy as np

st.set_option('deprecation.showfileUploaderEncoding', False)
def import_and_predict(image_data, model):
    
        size = (64,64)    
        image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
        image = np.asarray(image)
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #img_resize = (cv2.resize(img, dsize=(75, 75),    interpolation=cv2.INTER_CUBIC))/255.
        
        img_reshape = img[np.newaxis,...]
    
        prediction = model.predict(img_reshape)
        
        return prediction
if file is None:
    st.text("Please upload an image file")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    predictions = import_and_predict(image, model)
    score = tf.nn.softmax(predictions[0])
    st.write(predictions)
    st.write(score)
    # print(
    # "This image most likely belongs to {} with a {:.2f} percent confidence."
    # .format(class_names[np.argmax(score)], 100 * np.max(score))
    # )
    predictions = import_and_predict(image, model)
    class_name = ['Chamomile','Dandelion', 'Rose', 'SunFlower', 'Tulip']
    string="This image likely is: "+class_name[np.argmax(predictions)]
    st.success(string)

Overwriting app.py
